This notebook is designed to generate the phasenet-tf configuration files used for relocated mantle wedge events.

In [2]:
import pandas as pd
from pathlib import Path

In [3]:
# id	timestamp	amp	type	prob	event_index	gamma_score
df=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/association/phasenet_tf_all_months.assignment.threshold08.csv",names=["id","date","time","amp","type","prob","event_index","gamma_score"],skiprows=1,sep="\s+")
df["timestamp"]=pd.to_datetime(df["date"]+"T"+df["time"])

In [4]:
df

,id,date,time,amp,type,prob,event_index,gamma_score,timestamp
0,II.MSVF..BH,2009-11-01,12:29:38.419900,0.000001,p,0.86,-1,-1.000000,2009-11-01 12:29:38.419900
1,II.MSVF..BH,2009-11-01,16:03:19.149900,0.000001,p,0.91,2745,0.044766,2009-11-01 16:03:19.149900
2,II.MSVF..BH,2009-11-01,20:23:03.249900,0.000001,p,0.97,17803,0.059080,2009-11-01 20:23:03.249900
3,II.MSVF..BH,2009-11-02,04:58:30.619900,0.000001,s,0.91,-1,-1.000000,2009-11-02 04:58:30.619900
4,II.MSVF..BH,2009-11-02,08:37:00.949900,0.000001,p,0.94,-1,-1.000000,2009-11-02 08:37:00.949900
...,...,...,...,...,...,...,...,...,...
406592,Z1.VAVP..BH,2010-11-01,15:57:44.640,0.000001,p,0.96,-1,-1.000000,2010-11-01 15:57:44.640000
406593,Z1.VAVP..BH,2010-11-01,15:59:39.440,0.000001,p,0.93,1901,0.072638,2010-11-01 15:59:39.440000
406594,Z1.VAVP..BH,2010-11-01,19:06:43.610,0.000001,s,0.86,3539,0.122701,2010-11-01 19:06:43.610000
406595,Z1.VAVP..BH,2010-11-01,23:08:51.040,0.000001,p,0.97,18186,0.156814,2010-11-01 23:08:51.040000


In [10]:
EVENT_ID=13877
# select df with event_index==EVENT_ID and type==p
df_event=df[(df["event_index"]==EVENT_ID) & (df["type"]=="p")]
# select df with event_index==EVENT_ID and type==s
df_event_s=df[(df["event_index"]==EVENT_ID) & (df["type"]=="s")]
# in df_event_s, remove rows with same id as in df_event
df_event_s=df_event_s[~df_event_s["id"].isin(df_event["id"])]
# combine df_event and df_event_s
df_event=pd.concat([df_event,df_event_s])

# create a new df with columns network,station,start_time,end_time
df_tosave=pd.DataFrame(columns=["network","station","start_time","end_time"])
df_tosave["station"]=df_event["id"].str.split(".",expand=True)[1]
df_tosave["network"]=df_event["id"].str.split(".",expand=True)[0]
df_tosave["start_time"]=df_event["timestamp"]+pd.Timedelta(minutes=-8.5)
df_tosave["end_time"]=df_event["timestamp"]+pd.Timedelta(minutes=51.5)

# save df_tosave to predict_config_files/EVENT_ID.csv, start_time and end_time are in format like 2019-01-01T00:00:00.000000
df_tosave["start_time"]=df_tosave["start_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
df_tosave["end_time"]=df_tosave["end_time"].dt.strftime("%Y-%m-%dT%H:%M:%S.%f")
df_tosave.to_csv(f"predict_config_files/{EVENT_ID}.csv",index=False)